# ENEM 2016 - predição da nota da prova de matemática

O desafio abaixo visa prever as notas da prova de matemática de determinados alunos participantes do ENEM 2016 a partir da seleção livre de atributos de dois datasets pré-existentes.

As informações fornecidas para o desafio foram divididas em dois grupos:

* **Treino** - 13.730 instâncias e 167 atributos
* **Teste** - 4.576 instâncias e 47 atributos

Observações com fins didáticos:
* **As bibliotecas serão importadas na medida em que forem sendo necessárias**
* **Não foi levado em conta o tuning do modelo de Machine Learning**

#### Importando a biblioteca Pandas para transformar os datasets em dataframes e manipulá-los

In [1]:
import pandas as pd

#### Lendo os dados de treino e teste e instanciando um dataframe vazio para a resposta do desafio

In [2]:
df_train = pd.read_csv('/home/igorvroberto/GitHub/Data-Science-Projects/Data-Science-Projects/ENEM-2016/data/train.csv')
df_test = pd.read_csv('/home/igorvroberto/GitHub/Data-Science-Projects/Data-Science-Projects/ENEM-2016/data/test.csv')
df_answer = pd.DataFrame()

#### Verificando a forma dos datasets de treino e teste

In [3]:
df_train.shape, df_test.shape

((13730, 167), (4576, 47))

#### Inserindo no dataset de resposta a coluna com o número de inscrição provinda do dataset de teste

In [4]:
df_answer['NU_INSCRICAO'] = df_test['NU_INSCRICAO']

#### Verificando se o dataset de teste advém do dataset de treino

In [5]:
print(set(df_test.columns).issubset(set(df_train.columns)))

True


### Feature Engineering

Como a intenção do desafio é prever uma variável contínua, deve ser adotado um modelo de regressão.

Com base nessa premissa e após analisar a biblioteca (documento que explica o que significa cada coluna), busca-se selecionar os atributos (features) mais adequados para uma melhor predição do modelo.

#### Selecionando apenas os atributos numéricos do dataset de teste


In [6]:
df_test = df_test.select_dtypes(include=['int64','float64'])

In [7]:
df_test.columns.unique()

Index(['CO_UF_RESIDENCIA', 'NU_IDADE', 'TP_COR_RACA', 'TP_NACIONALIDADE',
       'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
       'IN_TREINEIRO', 'TP_DEPENDENCIA_ADM_ESC', 'IN_BAIXA_VISAO',
       'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DISLEXIA', 'IN_DISCALCULIA',
       'IN_SABATISTA', 'IN_GESTANTE', 'IN_IDOSO', 'TP_PRESENCA_CN',
       'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'NU_NOTA_CN', 'NU_NOTA_CH',
       'NU_NOTA_LC', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'NU_NOTA_REDACAO'],
      dtype='object')

#### Analisando a correlação entre as colunas que PROVAVELMENTE fazem mais sentido (questão subjetiva)

In [8]:
escolha_features = ['NU_IDADE',
                    'NU_NOTA_CN',
                    'NU_NOTA_CH',
                    'NU_NOTA_LC',
                    'NU_NOTA_REDACAO'
                   ]

In [9]:
df_test[escolha_features].corr()

,NU_IDADE,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_REDACAO
NU_IDADE,1.000000,-0.105278,-0.048229,-0.079171,-0.161202
NU_NOTA_CN,-0.105278,1.000000,0.598574,0.545801,0.442692
NU_NOTA_CH,-0.048229,0.598574,1.000000,0.679993,0.537141
NU_NOTA_LC,-0.079171,0.545801,0.679993,1.000000,0.495745
NU_NOTA_REDACAO,-0.161202,0.442692,0.537141,0.495745,1.000000


Como a coluna "NU_IDADE" possui uma baixa correlação negativa com os demais atributos, será desconsiderada

In [10]:
features = [
            'NU_NOTA_CN',
            'NU_NOTA_CH',
            'NU_NOTA_LC',
            'NU_NOTA_REDACAO'
           ]

#### Verificando valores nulos dos dados de treino e teste

In [11]:
df_train[features].isnull().sum(), df_test[features].isnull().sum()

(NU_NOTA_CN         3389
 NU_NOTA_CH         3389
 NU_NOTA_LC         3597
 NU_NOTA_REDACAO    3597
 dtype: int64,
 NU_NOTA_CN         1134
 NU_NOTA_CH         1134
 NU_NOTA_LC         1199
 NU_NOTA_REDACAO    1199
 dtype: int64)

### Observação
Com os valores nulos, é necessário tomar alguns cuidados que serão explicados conforme abaixo:

1) A ordem dos datasets é treino > teste > resposta, portanto não se deve utilizar a função **dropna** para a remoção das linhas com valores nulos de forma a evitar posterior erro no preenchimento do dataset de resposta (haverá divergência no número de linhas entre os datasets de teste e de resposta).

2) Existem duas possibilidades: substituir os valores nulos pela média das notas de cada prova ou alterá-los para 0 (valores nulos incluem **NaN**). Entre as duas opções, o modelo desempenhou melhor performance com a segunda. 

In [12]:
for c in features:
    df_train[c].fillna(0, inplace=True)
    df_test[c].fillna(0, inplace=True)
    df_train['NU_NOTA_MT'].fillna(0, inplace=True)

#### Instanciando os dados de treino e teste

In [13]:
X_train = df_train[features]
y_train = df_train['NU_NOTA_MT']

X_test = df_test[features]

#### Importando o modelo de regressão e treinando-o

In [14]:
from sklearn.ensemble import RandomForestRegressor

mdl=RandomForestRegressor()
mdl.fit(X_train, y_train)

RandomForestRegressor()

#### Predizendo os valores das notas de matemática

In [15]:
y_pred = mdl.predict(X_test)
y_pred

array([413.001, 435.138, 601.828, ..., 691.664, 460.034,   0.   ])

#### Verificando as métricas R2, MAE, MSE e RMSE

In [16]:
from sklearn import metrics
import numpy as np

pred_train = mdl.predict(X_train)

print('R2:', np.around((metrics.r2_score(y_train, pred_train)),3))
print('MAE:', np.around((metrics.mean_absolute_error(y_train, pred_train)),3))
print('MSE:', np.around((metrics.mean_squared_error(y_train, pred_train)),3))
print('RMSE:', np.around(np.sqrt(metrics.mean_squared_error(y_train, pred_train)),3))

R2: 0.988
MAE: 16.708
MSE: 618.729
RMSE: 24.874


#### Inserindo a coluna com as notas de matemática preditas no dataset de resposta

In [17]:
df_answer['NU_NOTA_MT'] = np.around(y_pred,2)

In [18]:
df_answer.head()

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,413.00
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,435.14
2,b38a03232f43b11c9d0788abaf060f7366053b6d,601.83
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,0.00
4,715494628a50142ce8cb17191cfe6d0f3cae0934,510.44


#### Salvando o arquivo .csv

In [ ]:
df_resposta.to_csv('answer.csv', index=False, header=True)